In [252]:
import datetime as datetime
import pandas as pd
import numpy as np
import datetime 
import seaborn as sns
import matplotlib.pyplot as plt

#Permite ver los graficos sin colocar plt.show
%matplotlib inline 

## Juntamos varios csv de properati para poder tener mas variedad de propiedades

In [254]:
propiedades2017 = pd.read_csv("properati-AR-2017-08-01-properties-sell.csv", low_memory = False)
propiedades2015 = pd.read_csv("properati-AR-2015-12-01-properties-sell.csv", low_memory = False)
frames = [propiedades2015, propiedades2017]
propiedades = pd.concat(frames)

In [255]:

def obtenerBarrio(row):
    try: 
        places = row['place_with_parent_names'].split('|')
        if (places[2] == 'Capital Federal' or places[4] == ''):
            if(places[3] == ''):
                return row['place_name']
            return places[3]
        return places[4]
    except:
        if(places[3] == ''):
            return row['place_name']
        return places[3]

In [256]:
#Dropeamos las propiedades cuyo ID este duplicado. Nos quedamos solo con las propiedades con año de 
#creacion 2014/2015/2016/2017
#Agregamos una columna que determina en que zona de BS AS se encuentra la propiedad
propiedades.drop_duplicates(subset=[ 'id'])
propiedades['created_on'] = pd.to_datetime(propiedades['created_on'])
propiedades = propiedades[propiedades.created_on.dt.year > 2013]
propiedades['zona'] = propiedades['place_with_parent_names'].apply(lambda x: x.split('|')[2])

### Filtrado de propiedades y analisis de la descripción

In [257]:
#Nos quedamos solamente con las propiedades que esten en Capital Federal o el Gran Buenos Aires.
dataCapGba = propiedades.loc[(propiedades.zona.str.contains('Capital Federal') | propiedades.state_name.str.contains('G.B.A') ),:]
dataCapGba = dataCapGba.loc[dataCapGba['description'].notnull(), :]
dataCapGba = dataCapGba.loc[dataCapGba['price_aprox_usd'] > 1000,:]
dataCapGba = dataCapGba.loc[dataCapGba['price_usd_per_m2'] > 100,:]
dataCapGba = dataCapGba.loc[dataCapGba['price_usd_per_m2'] < 10000,:]
dataCapGba = dataCapGba.loc[dataCapGba['surface_total_in_m2'] > 20, :]
dataCapGba['barrio'] = dataCapGba.apply(obtenerBarrio, axis =1)
dataCapGba['barrio'] = dataCapGba['barrio'].apply(lambda x : x.strip())
dataCapGba = dataCapGba.loc[dataCapGba['barrio'] != '', :]

#Analizamos la descripcion para obtener informacion que nos puede ser util a la hora de aproximar el
#valor de la propiedad. Como por ej: Si tiene cochera o no.
dataCapGba['pileta'] = dataCapGba.description.str.contains('pileta')
dataCapGba['cochera'] = dataCapGba.description.str.contains('cochera')
dataCapGba['aire acondicionado'] = dataCapGba.description.str.contains('aire acondicionado')
dataCapGba['seguridad'] = dataCapGba.description.str.contains('seguridad')
dataCapGba['patio'] = dataCapGba.description.str.contains('patio')
dataCapGba['amenities'] = dataCapGba.description.str.contains('ameniti') | dataCapGba.description.str.contains('SUM') | dataCapGba.description.str.contains('sauna') 
dataCapGba['parrilla'] = dataCapGba.description.str.contains('parrilla') | dataCapGba.description.str.contains('asador')

#En caso de que diga algo en la descripcion, se le pone un 1. Si no dice, se le pone 0.
dataCapGba['pileta'] = dataCapGba['pileta'].map(lambda x: 1 if x== True  else 0 )
dataCapGba['cochera'] = dataCapGba['cochera'].map(lambda x: 1 if x==True  else 0)
dataCapGba['aire acondicionado'] = dataCapGba['aire acondicionado'].map(lambda x: 1 if x==True  else 0)
dataCapGba['seguridad'] = dataCapGba['seguridad'].map(lambda x: 1 if x==True  else 0)
dataCapGba['patio'] = dataCapGba['patio'].map(lambda x: 1 if x==True  else 0)
dataCapGba['amenities'] = dataCapGba['amenities'].map(lambda x: 1 if x==True  else 0)
dataCapGba['parrilla'] = dataCapGba['parrilla'].map(lambda x: 1 if x==True  else 0)

In [258]:
dataCapGba = dataCapGba.loc[:, ['place_with_parent_names','created_on', 'property_type', 'lat', 'lon', 'barrio','price_aprox_usd', 'price_usd_per_m2', 'surface_total_in_m2', 'surface_covered_in_m2', 'zona', 'pileta', 'cochera', 'aire acondicionado', 'seguridad', 'patio', 'amenities' ]]

In [259]:
#Cargo el csv que tenemos que utilizar para aproximar el valor de la propiedad.
dataNo_price = pd.read_csv('properati_dataset_testing_noprice.csv')

In [260]:
#Modificamos el dataframe para agregarle todas las columnas nuevas que vamos a utilizar.
dataNo_price['created_on'] = pd.to_datetime(dataNo_price['created_on'])
dataNo_price['zona'] = dataNo_price['place_with_parent_names'].apply(lambda x: x.split('|')[2])
dataNo_price['barrio'] = dataNo_price.apply(obtenerBarrio, axis =1)
dataNo_price['barrio'] = dataNo_price['barrio'].apply(lambda x : x.strip())

dataNo_price['pileta'] = dataNo_price.description.str.contains('pileta')
dataNo_price['cochera'] = dataNo_price.description.str.contains('cochera')
dataNo_price['aire acondicionado'] = dataNo_price.description.str.contains('aire acondicionado')
dataNo_price['seguridad'] = dataNo_price.description.str.contains('seguridad')
dataNo_price['patio'] = dataNo_price.description.str.contains('patio')
dataNo_price['amenities'] = dataNo_price.description.str.contains('ameniti') | dataNo_price.description.str.contains('SUM') | dataNo_price.description.str.contains('sauna') 
dataNo_price['parrilla'] = dataNo_price.description.str.contains('parrilla') | dataNo_price.description.str.contains('asador')

dataNo_price['pileta'] = dataNo_price['pileta'].map(lambda x: 1 if x== True  else 0 )
dataNo_price['cochera'] = dataNo_price['cochera'].map(lambda x: 1 if x==True  else 0)
dataNo_price['aire acondicionado'] = dataNo_price['aire acondicionado'].map(lambda x: 1 if x==True  else 0)
dataNo_price['seguridad'] = dataNo_price['seguridad'].map(lambda x: 1 if x==True  else 0)
dataNo_price['patio'] = dataNo_price['patio'].map(lambda x: 1 if x==True  else 0)
dataNo_price['amenities'] = dataNo_price['amenities'].map(lambda x: 1 if x==True  else 0)
dataNo_price['parrilla'] = dataNo_price['parrilla'].map(lambda x: 1 if x==True  else 0)

In [261]:
dataNo_price = dataNo_price.loc[:, ['id','place_with_parent_names','created_on', 'property_type', 'lat', 'lon', 'barrio', 'surface_total_in_m2', 'surface_covered_in_m2', 'zona', 'pileta', 'cochera', 'aire acondicionado', 'seguridad', 'patio', 'amenities' ]]

In [262]:
#Funciones que se fijan que no haya propiedades que tengan latitud o longitud fuera de los rangos que tienen la mayoria.
def corregirLatitud(lat):
    if np.isnan(lat): 
        return lat
    if lat > -34 or lat < -35.2:
        lat = float('nan')
    return lat

def  corregirLongitud(lon):
    if np.isnan(lon): 
        return lon
    if lon > -57.7 or lon < -59:
        lon = float('nan') 
    return lon

In [263]:
dataCapGba['lat'] = dataCapGba['lat'].apply(corregirLatitud)
dataCapGba['lon'] = dataCapGba['lon'].apply(corregirLongitud)
dataNo_price['lat'] = dataNo_price['lat'].apply(corregirLatitud)
dataNo_price['lon'] = dataNo_price['lon'].apply(corregirLongitud)

In [264]:
dicc_cambio_Propiedad = {'departamento': 'apartment', 'casa': 'house', 'ph': 'PH'}
dataNo_price['property_type'] = dataNo_price['property_type'].apply(lambda x: dicc_cambio_Propiedad.get(x))

In [265]:
#Funcion que elimina los Nan de las columnas surface_total y surface_covered
def recuperarSuperficieTotal(row):
    if (np.isnan(row['surface_total_in_m2'])):
        if (row['property_type'] == 'apartment') & (not(np.isnan(row['surface_covered_in_m2']))):
            row['surface_total_in_m2'] = row['surface_covered_in_m2']
    return row['surface_total_in_m2']        

In [266]:
dataNo_price['surface_total_in_m2'] = dataNo_price.apply(recuperarSuperficieTotal, axis = 1)

In [267]:
#Junto ambos csvs para poder crear diccionarios para surface_total, surface_covered, lat y lon. Esto es para rellenar
#los registros que tengan valor Nan
concatenado = pd.concat([dataCapGba, dataNo_price], axis=0, ignore_index=True)
concatenado.dropna(inplace=True, subset=['surface_covered_in_m2', 'surface_total_in_m2'], how='all')
concatenado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105289 entries, 0 to 106062
Data columns (total 18 columns):
aire acondicionado         105289 non-null int64
amenities                  105289 non-null int64
barrio                     105289 non-null object
cochera                    105289 non-null int64
created_on                 105289 non-null datetime64[ns]
id                         13392 non-null float64
lat                        71444 non-null float64
lon                        71426 non-null float64
patio                      105289 non-null int64
pileta                     105289 non-null int64
place_with_parent_names    105289 non-null object
price_aprox_usd            91897 non-null float64
price_usd_per_m2           91897 non-null float64
property_type              105289 non-null object
seguridad                  105289 non-null int64
surface_covered_in_m2      99674 non-null float64
surface_total_in_m2        104592 non-null float64
zona                       105289 no

In [268]:
lat_barrio = concatenado.loc[:,['lat', 'barrio']]
lat_barrio = lat_barrio.groupby('barrio').agg('mean')
dicc_lat = {}
for x in range(len(lat_barrio)):
    dicc_lat[lat_barrio.index[x]] = lat_barrio.lat[x]
dicc_lat['Buenos Aires Interior'] = dicc_lat['Palermo']
dicc_lat['Altos de Hudson I'] = dicc_lat['Altos de Hudson II']
dicc_lat['Sourigues'] = -34.805492
dicc_lat['Terralagos'] = -34.907032
dicc_lat['Bs.As. G.B.A. Zona Sur'] = -34.874153

In [269]:
lon_barrio = concatenado.loc[:,['lon', 'barrio']]
lon_barrio = lon_barrio.groupby('barrio').agg('mean')
dicc_lon = {}
for x in range(len(lon_barrio)):
    dicc_lon[lon_barrio.index[x]] = lon_barrio.lon[x]
dicc_lon['Buenos Aires Interior'] = dicc_lon['Palermo']
dicc_lon['Altos de Hudson I'] = dicc_lon['Altos de Hudson II']
dicc_lon['Sourigues'] = -58.222173
dicc_lon['Terralagos'] = -58.514642
dicc_lon['Bs.As. G.B.A. Zona Sur'] = -57.883674
dicc_lon['Haras San Pablo'] = -59.0397703

In [270]:
superficie_covered_barrio = concatenado.loc[:, ['barrio','property_type', 'surface_covered_in_m2']]
superficie_covered_barrio = superficie_covered_barrio.groupby(['barrio', 'property_type']).agg(np.mean)
dicc_superficie_covered = {}
for x in range(len(superficie_covered_barrio)):
    dicc_superficie_covered[superficie_covered_barrio.index[x]] = superficie_covered_barrio.surface_covered_in_m2[x]
dicc_superficie_covered[('Buenos Aires Interior', 'apartment' )] = dicc_superficie_covered[('Palermo', 'apartment')]
dicc_superficie_covered[('Villa Numancia', 'house')] = 300
dicc_superficie_covered[('Villa Celina', 'house')] = 100
dicc_superficie_covered[('Villa Riachuelo', 'apartment')] = 65

In [271]:
superficie_total_barrio = concatenado.loc[:, ['barrio','property_type', 'surface_total_in_m2']]
superficie_total_barrio = superficie_total_barrio.groupby(['barrio', 'property_type']).agg(np.mean)
dicc_superficie_total = {}
for x in range(len(superficie_total_barrio)):
    dicc_superficie_total[superficie_total_barrio.index[x]] = superficie_total_barrio.surface_total_in_m2[x]

dicc_superficie_total[('Buenos Aires Interior', 'apartment' )] = dicc_superficie_total[('Palermo', 'apartment')]
dicc_superficie_total[('Villa Numancia', 'house')] = 500
dicc_superficie_total[('Altos de Hudson I', 'house')] = dicc_superficie_total[('Altos de Hudson II', 'house')]

In [272]:
def llenadoDeSuperficieTotal(reg):
    
    if reg['surface_total_in_m2'] == 0 or np.isnan(reg['surface_total_in_m2']):
        reg['surface_total_in_m2'] = dicc_superficie_total[(reg['barrio'], reg['property_type'])]
    return reg['surface_total_in_m2']

def llenadoDeSuperficieCovered(reg):
    
    if reg['surface_covered_in_m2'] == 0 or np.isnan(reg['surface_covered_in_m2']):
        reg['surface_covered_in_m2'] = dicc_superficie_covered[(reg['barrio'], reg['property_type'])]
    return reg['surface_covered_in_m2']

def llenadoDeLatitud(reg):
    
    if np.isnan(reg['lat']):
        reg['lat'] = dicc_lat[reg['barrio']] 
    return reg['lat']

def llenadoDeLongitud(reg):
    
    if np.isnan(reg['lon']):
        reg['lon'] = dicc_lon[reg['barrio']] 
    return reg['lon']


In [273]:
#Aplico todas las funciones al csv de properati para que no me quede ninguna superficie, lat o lon en Nan
dataNo_price['lat'] = dataNo_price.apply(llenadoDeLatitud, axis=1)
dataNo_price['lon'] = dataNo_price.apply(llenadoDeLongitud, axis=1)
dataNo_price['surface_total_in_m2'] = dataNo_price.apply(llenadoDeSuperficieTotal, axis=1)
dataNo_price['surface_covered_in_m2'] = dataNo_price.apply(llenadoDeSuperficieCovered, axis=1)
dataNo_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 16 columns):
id                         14166 non-null int64
place_with_parent_names    14166 non-null object
created_on                 14166 non-null datetime64[ns]
property_type              14166 non-null object
lat                        14166 non-null float64
lon                        14166 non-null float64
barrio                     14166 non-null object
surface_total_in_m2        14166 non-null float64
surface_covered_in_m2      14166 non-null float64
zona                       14166 non-null object
pileta                     14166 non-null int64
cochera                    14166 non-null int64
aire acondicionado         14166 non-null int64
seguridad                  14166 non-null int64
patio                      14166 non-null int64
amenities                  14166 non-null int64
dtypes: datetime64[ns](1), float64(4), int64(7), object(4)
memory usage: 1.7+ MB


In [274]:
#Hago lo mismo para el csv que armamos para entrar los algoritmos de machine learning
dataCapGba['lat'] = dataCapGba.apply(llenadoDeLatitud, axis=1)
dataCapGba['lon'] = dataCapGba.apply(llenadoDeLongitud, axis=1)
dataCapGba['surface_total_in_m2'] = dataCapGba.apply(llenadoDeSuperficieTotal, axis=1)
dataCapGba['surface_covered_in_m2'] = dataCapGba.apply(llenadoDeSuperficieCovered, axis=1)
dataCapGba.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91897 entries, 3658 to 187478
Data columns (total 17 columns):
place_with_parent_names    91897 non-null object
created_on                 91897 non-null datetime64[ns]
property_type              91897 non-null object
lat                        91838 non-null float64
lon                        91836 non-null float64
barrio                     91897 non-null object
price_aprox_usd            91897 non-null float64
price_usd_per_m2           91897 non-null float64
surface_total_in_m2        91897 non-null float64
surface_covered_in_m2      91860 non-null float64
zona                       91897 non-null object
pileta                     91897 non-null int64
cochera                    91897 non-null int64
aire acondicionado         91897 non-null int64
seguridad                  91897 non-null int64
patio                      91897 non-null int64
amenities                  91897 non-null int64
dtypes: datetime64[ns](1), float64(6), int64(6

In [275]:
dataAExportarTrain = dataCapGba.loc[dataCapGba['lat'].notnull() & dataCapGba['lon'].notnull() & dataCapGba['surface_covered_in_m2'].notnull() ]

In [276]:
#Formulas importantes 

#Funcion para calcular distancias entre coordenadas (lat, lon)
from math import sin,cos,sqrt,asin,pi

#Formula del Harvesine sacada de
#https://joseguerreroa.wordpress.com/2012/11/13/distancia-entre-dos-puntos-de-la-superficie-terrestre-mediante-la-formula-de-haversine-con-python/
#coord = (lat, long)
def distancia(coord1, coord2):
    rTierra = 6374000 #metros
    c = pi/180 #para pasar a raddianes
    d = 2*rTierra*asin(sqrt(sin(c*(coord2[0]-coord1[0])/2)**2 + cos(c*coord1[0])*cos(c*coord2[0])*sin(c*(coord2[1]-coord1[1])/2)**2))
    return d

def obtener_geoPos(reg):
    lat, lon = reg
    return lat,lon

def obtener_distancia_al_centro(geoPos):
    centro = (-34.608056,  -58.370278)# Se utilizo las coordenadas de la Casa Rosada
    return distancia(geoPos, centro)
#calcula la distancia en metros entre la propiedad y la villa Fuerte Apache. Recibe por parametro un registro
def distanciaAFuerteApache(geoPos):
      #Ubicacion
    fuerteApache = (-34.622374, -58.539921) #Tomamos este como el punto central de la villa
    return distancia(geoPos, fuerteApache)

#calcula la distancia en metros entre la propiedad y la villa Carlos Gardel. Recibe por parametro un registro
def distanciaACarlosGardel(geoPos):
    #Ubicacion
    carlosGardel = (-34.6244, -58.5778) #Tomamos este como el punto central de la villa
    return distancia(geoPos, carlosGardel)

#Informacion sacada de 
#http://blogs.monografias.com/estampas-de-buenos-aires/2015/05/21/las-villas-de-emergencia-en-la-ciudad/
def distanciaAVillas(geoPos):
    
    Indoamericano = (-34.665202, -58.468526)# Se tomaron estas coordenadas ya que alrededor hay una gran cantidad de villas
    return distancia(geoPos, Indoamericano)

In [277]:
#Coordenadas de todas las estaciones de subtes separada cada una por su respectiva linea
def obtener_subtes_cercanos(geoPos):
    lineaE = [(-34.609563795, -58.374052048), (-34.612989872, -58.377828598),(-34.618217016, -58.381583691), (-34.622366711, -58.385403156),
              (-34.622455000, -58.391432762), (-34.623125994, -58.397119045), (-34.623867613, -58.402848244), (-34.624679854, -58.409628868),
              (-34.625545060, -58.416645527), (-34.626798708, -58.426151276), (-34.628069994, -58.434305191), (-34.631618899, -58.442544937),
              (-34.636385845, -58.450076580), (-34.639563657, -58.457865715), (-34.642829615, -58.461534977)]
    
    lineaA = [(-34.608557137, -58.374567032), (-34.609086958, -58.382549286), (-34.609404849, -58.386669159), (-34.609652097, -58.398342133),
              (-34.609687418, -58.406839371), (-34.610782364, -58.415164948), (-34.615091365, -58.429327011), (-34.611736015, -58.421730995),
              (-34.618022770, -58.435893059), (-34.620459643, -58.441429138), (-34.623567434, -58.448810577), (-34.626533854, -58.456192017)]
    
    lineaB = [(-34.602870174, -58.369545937), (-34.603294060, -58.374567032), (-34.604141827, -58.386712074), (-34.604530384, -58.392891884),
              (-34.604530384, -58.399200439), (-34.604565707, -58.405079842), (-34.604106503, -58.411602974), (-34.603152765, -58.421001434),
              (-34.602057719, -58.430914879), (-34.599055099, -58.439583778), (-34.591671724, -58.447651863), (-34.584111033, -58.466534615),
              (-34.581567095, -58.473873138), (-34.573828804, -58.487434387)]
    
    lineaC = [(-34.590505868, -58.376069069), (-34.595522465, -58.377442360), (-34.602022395, -58.378129005), (-34.608980994, -58.380703926),
              (-34.612407095, -58.380532265), (-34.618058088, -58.380274773), (-34.622331395, -58.379931450), (-34.627964055, -58.381175995)]
    
    lineaD = [(-34.607621111, -58.374266624), (-34.604336105, -58.380360603), (-34.601845773, -58.385360241), (-34.599426017, -58.392827511),
              (-34.599531993, -58.397955894), (-34.594497974, -58.402633667), (-34.591636395, -58.407139778), (-34.588244767, -58.411281109),
              (-34.585170985, -58.415830135), (-34.578440064, -58.426043987), (-34.575295247, -58.434326649), (-34.570242091, -58.444347382),
              (-34.566125132, -58.452243805), (-34.562396722, -58.456299305), (-34.555982053, -58.462092876)]
    
    lineaH = [(-34.588138777, -58.398170471), (-34.598684179, -58.403813839), (-34.603700283, -58.405380249), (-34.608945673, -58.405938148),
              (-34.615515189, -58.404779434), (-34.622896444, -58.402483463), (-34.629341261, -58.400981426), (-34.635255927, -58.399178982),
              (-34.637515748, -58.405981064), (-34.641796880, -58.412622213)]
    
    subtes = lineaA + lineaB + lineaC + lineaD + lineaE + lineaH
    cant_subtes_cercanos = 0
    
    for i in range(len(subtes)):
        if (distancia(geoPos, subtes[i]) <= 700):
            cant_subtes_cercanos += 1
    return cant_subtes_cercanos

In [278]:
#Coordenadas de estaciones de trenes separadas por su respectiva linea.
def obtener_distancias_a_trenes(geoPos):
    lineaSanMartin = [(-34.590505868, -58.376069069), (-34.578987744, -58.426752090), (-34.592448952, -58.446493149),
                      (-34.596794231, -58.466062546), (-34.601492528, -58.494043350), (-34.602446285, -58.512883186),
                      (-34.603364708, -58.527774811), (-34.603788592, -58.540949821), (-34.604671676, -58.564724922),
                      (-34.604848292, -58.596138954), (-34.590965146, -58.632230759), (-34.578563734, -58.656563759),
                      (-34.563722005, -58.684630394), (-34.551316528, -58.703513145), (-34.544070309, -58.713254929),
                      (-34.519074848, -58.750634193), (-34.503798116, -58.797540665), (-34.490844294, -58.839511871),
                      (-34.478719817, -58.877105713), (-34.468530709, -58.908262253)]
    
    lineaUrquiza = [(-34.586796217, -58.455419540), (-34.590081916, -58.466577530), (-34.592378295, -58.476104736),
                    (-34.592961212, -58.494215012), (-34.594409655, -58.502969742), (-34.595557792, -58.510587215),
                    (-34.597218148, -58.524470329), (-34.595469475, -58.535284996), (-34.593773756, -58.547022343),
                    (-34.591689388, -58.559403419), (-34.589958263, -58.571977615), (-34.588933703, -58.590688705), 
                    (-34.588421418, -58.607017994), (-34.587767808, -58.620772362), (-34.587608821, -58.629484177),
                    (-34.580136095, -58.640384674), (-34.573369430, -58.645083904), (-34.562856156, -58.659975529),
                    (-34.555593269, -58.669481277), (-34.549089709, -58.678472042), (-34.543292628, -58.686540127),
                    (-34.536575994, -58.695423603), (-34.532846260, -58.700573444)]
    
    lineaRoca = [(-34.625929, -58.381537), (-34.653588, -58.378764), (-34.661962, -58.376580), (-34.686132, -58.383069), 
                 (-34.707708, -58.391056), (-34.726060, -58.390585), (-34.743153, -58.395465), (-34.761184, -58.397406), 
                 (-34.775976, -58.396176), (-34.795120, -58.408172), (-34.796928, -58.428929), (-34.803084, -58.448641),
                 (-34.814747, -58.469670), (-34.833361, -58.496412), (-34.854207, -58.522616), (-34.678947, -58.345075),
                 (-34.691431, -58.325349), (-34.697429, -58.311186), (-34.703336, -58.296038), (-34.709460, -58.280410),
                 (-34.724708, -58.260829), (-34.751987, -58.234202), (-34.763887, -58.208775), (-34.775144, -58.194796),
                 (-34.789613, -58.203033), (-34.807186, -58.213065), (-34.819260, -58.230481)]
    
    lineaMitre = [(-34.589495900, -58.375815000), (-34.571805900, -58.425271100), (-34.575154900, -58.432836300), 
                  (-34.573093100, -58.448278700), (-34.567728900, -58.461285900), (-34.568478300, -58.479030100), 
                  (-34.574920000, -58.487589000), (-34.580332000, -58.501930000), (-34.582301700, -58.517546400), 
                  (-34.573738900, -58.531764100), (-34.563129900, -58.540566100), (-34.557439000, -58.547953800), 
                  (-34.550860200, -58.556444200), (-34.542311700, -58.567056900), (-34.534233500, -58.576114100),
                  (-34.565444000, -58.475835000), (-34.554347600, -58.486654600), (-34.542106100, -58.494398600),
                  (-34.529326200, -58.494565900), (-34.523137200, -58.494077700), (-34.514509800, -58.489039400),
                  (-34.562738100, -58.435664200), (-34.558524000, -58.449531000), (-34.548763000, -58.463060000),
                  (-34.536155000, -58.468273900), (-34.524049100, -58.473396200), (-34.508383800, -58.480840100),
                  (-34.497587000, -58.488880900), (-34.487893200, -58.497004400), (-34.479836700, -58.503698100),
                  (-34.471657700, -58.514076400), (-34.460823300, -58.526215300), (-34.455746600, -58.540753700),
                  (-34.450016200, -58.551330000), (-34.444265500, -58.558544900), (-34.436394400, -58.568376300),
                  (-34.423474500, -58.581848200)]
    
    lineaBelgranoSur = [(-34.712196387, -58.524127007), (-34.710679445, -58.537001610), (-34.708809687, -58.551721573),
                        (-34.704434996, -58.587427139), (-34.699566116, -58.626222610), (-34.696849297, -58.647508621),
                        (-34.691927041, -58.686497211), (-34.708986081, -58.731236458), (-34.722249832, -58.530778885),
                        (-34.750622175, -58.584401608), (-34.756387146, -58.601760864), (-34.760230237, -58.615579605),
                        (-34.771652680, -58.646779060), (-34.645424635, -58.393964767), (-34.648884536, -58.415679932),
                        (-34.662228516, -58.440999985), (-34.665334743, -58.448553085), (-34.677299687, -58.477134705),
                        (-34.683757921, -58.491854668), (-34.689968643, -58.507561684), (-34.702318127, -58.511638641),
                        (-34.710714723, -58.514385223), (-34.710291385, -58.505930901), (-34.718122776, -58.477563858),
                        (-34.718934094, -58.462457657), (-34.704928924, -58.440527916), (-34.697801958, -58.429198265),
                        (-34.681322908, -58.428468704), (-34.661028352, -58.415594101)]
    
    lineaBelgranoNorte = [(-34.575254, -58.399343), (-34.543440, -58.447546), (-34.537156, -58.477231), 
                          (-34.543428, -58.500637), (-34.537072, -58.513160), (-34.531088, -58.524844), 
                          (-34.528640, -58.537715), (-34.523648, -58.546230), (-34.508921, -58.564748),
                          (-34.504075, -58.620185), (-34.502567, -58.641925), (-34.502914, -58.666516),
                          (-34.503488, -58.680711), (-34.498965, -58.694541), (-34.493690, -58.706301),
                          (-34.485394, -58.725105), (-34.478762, -58.739754), (-34.470653, -58.757971),
                          (-34.463085, -58.774975), (-34.453257, -58.796700), (-34.416597, -58.871704)]
    
    trenes = lineaSanMartin + lineaUrquiza + lineaRoca + lineaMitre + lineaBelgranoSur + lineaBelgranoNorte
    cant_trenes_cercanos = 0
    
    for i in range(len(trenes)):
        if (distancia(geoPos, trenes[i]) <= 800):
            cant_trenes_cercanos += 1
    return cant_trenes_cercanos

In [279]:
#funcion para obtener la suma de estaciones de trenes y estaciones de subtes cercanas a una propiedad.
def obtener_distancia_subteytren(geoPos):
    return obtener_subtes_cercanos(geoPos) + obtener_distancias_a_trenes(geoPos)


In [280]:
def obtenerDistanciaZonaInsegura(geoPos):
    #Obtenemos las coordenadas de las comisarias mas afectadas.
    comisarias = [(-34.630975500, -58.458303800), (-34.602049200, -58.373201600), (-34.629577700, -58.524118200), 
                  (-34.560070200, -58.456198000), (-34.661963300, -58.501216900), (-34.604256000, -58.415567000), 
                  (-34.626941500, -58.447969600)]
    
    distancias = [obtenerDistanciaDelitoMenor(geoPos), obtenerDistanciaDelitoGrave(geoPos)] 
    for i in range(len(comisarias)):
        distancias.append(distancia(geoPos, comisarias[i]))
       
    return min( distancias)

def obtenerDistanciaDelitoMenor(geoPos):
    #Obtenenemos las coordenadas de los puntos donde se concentran la mayor cantidad de delitos.
    delitosMenores = [(-34.592166325, -58.373794556), (-34.584245294, -58.390552998), (-34.598480800, -58.403680000), 
                      (-34.535819454, -58.466749191), (-34.539142518, -58.475718498), (-34.562117527, -58.456835747),
                      (-34.570174948, -58.444776535), (-34.563255508, -58.438510895), (-34.577630903, -58.426966667),
                      (-34.581694293, -58.420443535), (-34.599066000, -58.382093800), (-34.617612000, -58.380684900)]
    distancias = [] 
    for i in range(len(delitosMenores)):
        distancias.append(distancia(geoPos, delitosMenores[i]))
    
    return min( distancias)

def obtenerDistanciaDelitoGrave(geoPos):
    delitosGraves = [(-34.635299800, -58.487350800), (-34.626672200, -58.456726200), (-34.630691600, -58.469656000),
                      (-34.617770100, -58.476078700), (-34.618856000, -58.403548700), (-34.610105500, -58.405879100), 
                      (-34.665207669, -58.469409943), (-34.637247398, -58.410530090)]
    distancias = [] 
    for i in range(len(delitosGraves)):
        distancias.append(distancia(geoPos, delitosGraves[i]))
    
    return min( distancias)

In [281]:
#Agrego varias columnas con distancias que nos pueden servir. Ademas la cantidad de subtes/trenes cercanos a cada propiedad.
dataAExportarTrain['geoPos'] = dataAExportarTrain.loc[:,['lat', 'lon']].apply(obtener_geoPos, axis = 1)
dataAExportarTrain['distanciaCentro'] = dataAExportarTrain['geoPos'].apply(obtener_distancia_al_centro)
dataAExportarTrain['distanciaAIndoamericano'] = dataAExportarTrain['geoPos'].apply(distanciaAVillas)
dataAExportarTrain['distanciaAFuerteApache'] = dataAExportarTrain['geoPos'].apply(distanciaAFuerteApache)
dataAExportarTrain['distanciaACarlosGardel'] = dataAExportarTrain['geoPos'].apply(distanciaACarlosGardel)
dataAExportarTrain['cant_subtes_cercanos'] = dataAExportarTrain['geoPos'].apply(obtener_subtes_cercanos)
dataAExportarTrain['cant_trenes_cercanos'] = dataAExportarTrain['geoPos'].apply(obtener_distancias_a_trenes)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [282]:
dataAExportarTrain = dataAExportarTrain.loc[dataAExportarTrain['distanciaCentro'] < 65000, :]
dataAExportarTrain = dataAExportarTrain.loc[dataAExportarTrain['surface_total_in_m2'] < 21000, :]
dataAExportarTrain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91796 entries, 3658 to 187478
Data columns (total 24 columns):
place_with_parent_names    91796 non-null object
created_on                 91796 non-null datetime64[ns]
property_type              91796 non-null object
lat                        91796 non-null float64
lon                        91796 non-null float64
barrio                     91796 non-null object
price_aprox_usd            91796 non-null float64
price_usd_per_m2           91796 non-null float64
surface_total_in_m2        91796 non-null float64
surface_covered_in_m2      91796 non-null float64
zona                       91796 non-null object
pileta                     91796 non-null int64
cochera                    91796 non-null int64
aire acondicionado         91796 non-null int64
seguridad                  91796 non-null int64
patio                      91796 non-null int64
amenities                  91796 non-null int64
geoPos                     91796 non-null obje

In [283]:
#Eliminamos las propiedades duplicadas
dataAExportarTrain.drop_duplicates(inplace=True, keep='first')

In [284]:
#Si tiene al menos 1 subte/tren cercano, se pone un 1 sino un 0.
dataAExportarTrain['subtes_cercanos'] = dataAExportarTrain['cant_subtes_cercanos'].apply(lambda x: 1 if x > 0 else 0)
dataAExportarTrain['trenes_cercanos'] = dataAExportarTrain['cant_trenes_cercanos'].apply(lambda x: 1 if x > 0 else 0)

In [285]:
dataAExportarTrain = dataAExportarTrain.drop(['place_with_parent_names', 'geoPos','cant_subtes_cercanos', 'cant_trenes_cercanos' ], axis = 1)

In [286]:
dataAExportarTrain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86846 entries, 3658 to 187478
Data columns (total 22 columns):
created_on                 86846 non-null datetime64[ns]
property_type              86846 non-null object
lat                        86846 non-null float64
lon                        86846 non-null float64
barrio                     86846 non-null object
price_aprox_usd            86846 non-null float64
price_usd_per_m2           86846 non-null float64
surface_total_in_m2        86846 non-null float64
surface_covered_in_m2      86846 non-null float64
zona                       86846 non-null object
pileta                     86846 non-null int64
cochera                    86846 non-null int64
aire acondicionado         86846 non-null int64
seguridad                  86846 non-null int64
patio                      86846 non-null int64
amenities                  86846 non-null int64
distanciaCentro            86846 non-null float64
distanciaAIndoamericano    86846 non-null flo

In [287]:
#Agregamos las distancias y la cant de trenes/subtes al csv de properati
dataAExportarTest = dataNo_price
dataAExportarTest['geoPos'] = dataAExportarTest.loc[:,['lat', 'lon']].apply(obtener_geoPos, axis = 1)
dataAExportarTest['distanciaCentro'] = dataAExportarTest['geoPos'].apply(obtener_distancia_al_centro)
dataAExportarTest['distanciaAIndoamericano'] = dataAExportarTest['geoPos'].apply(distanciaAVillas)
dataAExportarTest['distanciaAFuerteApache'] = dataAExportarTest['geoPos'].apply(distanciaAFuerteApache)
dataAExportarTest['distanciaACarlosGardel'] = dataAExportarTest['geoPos'].apply(distanciaACarlosGardel)
dataAExportarTest['cant_subtes_cercanos'] = dataAExportarTest['geoPos'].apply(obtener_subtes_cercanos)
dataAExportarTest['cant_trenes_cercanos'] = dataAExportarTest['geoPos'].apply(obtener_distancias_a_trenes)

In [288]:
dataAExportarTest['subtes_cercanos'] = dataAExportarTest['cant_subtes_cercanos'].apply(lambda x: 1 if x > 0 else 0)
dataAExportarTest['trenes_cercanos'] = dataAExportarTest['cant_trenes_cercanos'].apply(lambda x: 1 if x > 0 else 0)

In [289]:
dataAExportarTest = dataAExportarTest.drop(['place_with_parent_names', 'geoPos','cant_subtes_cercanos', 'cant_trenes_cercanos' ], axis = 1)

In [290]:
dataAExportarTest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 21 columns):
id                         14166 non-null int64
created_on                 14166 non-null datetime64[ns]
property_type              14166 non-null object
lat                        14166 non-null float64
lon                        14166 non-null float64
barrio                     14166 non-null object
surface_total_in_m2        14166 non-null float64
surface_covered_in_m2      14166 non-null float64
zona                       14166 non-null object
pileta                     14166 non-null int64
cochera                    14166 non-null int64
aire acondicionado         14166 non-null int64
seguridad                  14166 non-null int64
patio                      14166 non-null int64
amenities                  14166 non-null int64
distanciaCentro            14166 non-null float64
distanciaAIndoamericano    14166 non-null float64
distanciaAFuerteApache     14166 non-null float64
d

In [291]:
#Creamos los csv ya listo para entrenar a los algoritmos y para testear que tan buenos son.
dataAExportarTrain.to_csv('setEntrenamiento.csv')
dataAExportarTest.to_csv('setTesteo.csv')